# Sea Surface Current Prediction using LSTM

This notebook outlines the process of loading, inspecting, and preprocessing sea surface current data (`u` and `v` components) for training an LSTM model. The goal is to predict sea surface currents for the next 24 hours based on historical data.

### Import Libraries

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import matplotlib.path as mpath
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout, InputLayer
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.losses import MeanSquaredError
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

### Load and Inspect Data

The initial step involves loading the dataset using xarray, which facilitates working with multi-dimensional arrays, and inspecting its structure to understand the available dimensions, coordinates, and variables.

In [ ]:
# Load the dataset
netcdf_file_path = "Data/24_months_SSC_Data.nc"

# Open the data
dataset = xr.open_dataset(netcdf_file_path)

# Print head of dataset
dataset.head()

### Convert the dataset to a pandas DataFrame

In [ ]:
# Convert the dataset to a pandas DataFrame
df = dataset.to_dataframe().reset_index()

# Drop columns from the DataFrame
df = df.drop(columns=['stdu', 'stdv', 'cov', 'velo', 'head'])

# Remove rows where 'u' and 'v' have NaN values
df = df.dropna(subset=['u', 'v'])

# Ensure no NaN values remain in the DataFrame
assert not df.isnull().values.any(), "There are still NaN values in the DataFrame"

# Convert the 'time' column to a datetime object
df.index = pd.to_datetime(df['time'], format='%d-%m-%Y %H:%M:%S')

df

### Geospatial Visualization
Define boundaries and coordinates for visualization

In [ ]:
# Define the boundaries for map
boundaries = {
    'min_lon': 14.15,  
    'max_lon': 14.81,  
    'min_lat': 35.79,  
    'max_lat': 36.3   
}

# Define the coordinates 
polygon_coordinates = [
    (14.6, 35.87), 
    (14.35, 36.01), 
    (14.35, 36.09), 
    (14.6, 36.09), 
    (14.6, 35.87)
]

# Create a plot with Cartopy
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent([boundaries['min_lon'], boundaries['max_lon'], boundaries['min_lat'], boundaries['max_lat']], crs=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Plotting all points within the boundary from the dataframe 'df'
ax.scatter(df['lon'], df['lat'], s=10, color='blue', marker='o', alpha=0.5, transform=ccrs.Geodetic())

# Create a red polygon and add it to the plot
red_polygon = mpatches.Polygon(polygon_coordinates, closed=True, edgecolor='red', facecolor='none', linewidth=3, transform=ccrs.Geodetic())
ax.add_patch(red_polygon)

# Gridlines and labels
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False

plt.show()

### Data Filtering 
Filter the DataFrame to include only data points within the specified polygon.

In [ ]:
# Convert the polygon coordinates to a Path object
polygon_path = mpath.Path(polygon_coordinates)

# Use the Path object to find points inside the polygon
inside_polygon = df.apply(lambda row: polygon_path.contains_point((row['lon'], row['lat'])), axis=1)
df_inside_polygon = df[inside_polygon]

# Create a plot with Cartopy
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent([boundaries['min_lon'], boundaries['max_lon'], boundaries['min_lat'], boundaries['max_lat']], crs=ccrs.PlateCarree())

# Add map features
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Plotting only the points within the polygon
ax.scatter(df_inside_polygon['lon'], df_inside_polygon['lat'], s=10, color='red', marker='x', alpha=0.5, transform=ccrs.Geodetic())

# Gridlines and labels
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False

plt.show()

### Data Preparation

In [ ]:
# Convert the polygon coordinates to a Path object
polygon_path = mpath.Path(polygon_coordinates)

# Prepare the coordinates as a list of tuples
coordinates = list(zip(df['lon'], df['lat']))

# Check if each coordinate is inside the polygon
inside_mask = [polygon_path.contains_point(coord) for coord in coordinates]

# Filter the DataFrame for points within the polygon
df_inside_polygon = df[inside_mask]

# Removing potential duplicates
df_inside_polygon = df_inside_polygon.drop_duplicates(subset=['lon', 'lat'])

# Count the number of points inside the polygon
num_points_inside_polygon = df_inside_polygon.shape[0]
print(f"Total number of points inside the polygon: {num_points_inside_polygon}\n")
print("Coordinates of the points inside the polygon:")

# Print out all of the coordinate pairs
coordinates_inside = df_inside_polygon[['lon', 'lat']].values
for lon, lat in coordinates_inside:
    print(f"({lon}, {lat})")

In [ ]:
# Assuming 'inside_mask' is a list of booleans indicating whether each point is inside the polygon
inside_mask = [polygon_path.contains_point((lon, lat)) for lon, lat in zip(df['lon'], df['lat'])]

# Convert 'inside_mask' to a pandas Series to use as a boolean indexer
inside_series = pd.Series(inside_mask, index=df.index)

# Filter the original DataFrame using the boolean Series
final_df = df[inside_series]

# Display the resulting DataFrame
final_df

### Exctract data only for a single pair of coordinates

In [ ]:
# Coordinates 
target_lat = 36.03409957885742
target_lon = 14.528599739074707

# Filter the DataFrame for the exact coordinates
df_point = final_df[(final_df['lat'] == target_lat) & (final_df['lon'] == target_lon)]

# Display the filtered DataFrame
df_point

In [ ]:
df1_u = df_point['u']

plt.figure(figsize=(10, 5))  
plt.plot(df1_u.index, df1_u, label='u component')  
plt.title('Sea Surface Current "u" Component Over Time')  
plt.xlabel('Time')  
plt.ylabel('u Component Value')  
plt.legend()  
plt.grid(True)  
plt.show()

### Convert the time series data into a format suitable for LSTM

In [ ]:
def df_to_X_y(df_u, df_v, window_size=5):
    df_u_as_np = df_u.to_numpy()
    df_v_as_np = df_v.to_numpy()
    X = [] 
    y = []
    for i in range(len(df_u_as_np) - window_size):
        # Create a combined feature for each time step in the window
        combined_features = [[df_u_as_np[i+j], df_v_as_np[i+j]] for j in range(window_size)]
        X.append(combined_features)
        # The target remains the 'u' value at the next time step
        label = df_u_as_np[i + window_size]
        y.append(label)
    return np.array(X), np.array(y)

window_size = 1
X, y = df_to_X_y(df_point['u'], df_point['v'], window_size=window_size)
X.shape, y.shape

### Split the data

In [ ]:
def train_val_test_split(X, y, train_size, val_size, test_size):
    # Calculate the indices for the end of each segment
    total_size = len(X)
    train_end = int(total_size * train_size)
    val_end = train_end + int(total_size * val_size)
    
    # Split the data
    X_train = X[:train_end]
    y_train = y[:train_end]
    
    X_val = X[train_end:val_end]
    y_val = y[train_end:val_end]
    
    X_test = X[val_end:]
    y_test = y[val_end:]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

train_size = 0.70
val_size = 0.15
test_size = 1 - (train_size + val_size)

X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(X, y, train_size, val_size, test_size)

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

### Define & Compile the Model

In [ ]:
# Define the GRU model
model = Sequential()
model.add(InputLayer((window_size, 2)))
model.add(GRU(64, return_sequences=True))  # Use one GRU layer with fewer units
model.add(Dropout(0.2))  # Regularization with dropout
model.add(GRU(32))  # Last GRU does not return sequences
model.add(Dropout(0.2))  # Regularization with dropout
model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.01)))  # Dense layer with L2 regularization
model.add(Dense(1, activation='linear'))

model.summary()

# Define ModelCheckpoint callback to save the best model
model_checkpoint = ModelCheckpoint(filepath="Data/GRU/Saved_Models/gru_single_u_target_model",
                     save_best_only=True,
                     monitor='val_loss',  
                     mode='min',
                     verbose=0)   

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=8,
                               restore_best_weights=True,
                               min_delta=0.0001,  
                               verbose=1)             

# Compile the model
model.compile(loss=MeanSquaredError(),
              optimizer=Adam(learning_rate=0.001))

### Train the Model

In [ ]:
# Fit the model with the training data and validate with the validation data
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=100,
                    callbacks=[model_checkpoint, early_stopping])

# Plotting the training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training vs Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Evaluation on Test Set

In [ ]:
# Load the model
model = load_model("Data/GRU/Saved_Models/gru_single_u_target_model")

# Make predictions on the test data
test_predictions = model.predict(X_test)

print("y_test shape:", y_test.shape)
print("test_predictions shape:", test_predictions.shape)

In [ ]:
# Calculate metrics
mse_u = mean_squared_error(y_test, test_predictions)
m_ae_u = mean_absolute_error(y_test, test_predictions)
r2_u = r2_score(y_test, test_predictions)
explained_variance_u = explained_variance_score(y_test, test_predictions)

# Output the error metrics
print("Test MSE for 'u' component:", mse_u)
print("Test MAE for 'u' component:", m_ae_u)
print("Test R-squared for 'u' component:", r2_u)
print("Test Explained Variance for 'u' component:", explained_variance_u)

In [ ]:
test_results = pd.DataFrame({
    'Test Predictions u': test_predictions.flatten(),
    'Actuals u': y_test
})

test_results

In [ ]:
# Plot for the 'u' component test predictions vs actuals
plt.figure(figsize=(10, 5))
plt.plot(test_results.index, test_results['Test Predictions u'], label='Test Predictions u', color='blue', linestyle='--')
plt.plot(test_results.index, test_results['Actuals u'], label='Actuals u', color='red', alpha=0.7)
plt.title('Test Predictions vs Actuals for u Component')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.legend()
plt.show()

### Making 24-Hour Predictions on the Test Set

In [ ]:
# Take the first 24 predictions for comparison
last_24_predictions = test_predictions[-24:]
last_24_actual = y_test[-24:]

# Flatten the predictions to make them a 1D array, if they're not already
first_24_predictions = last_24_predictions.flatten()

# Calculate metrics for the first 24-hour predictions
mse_24h = mean_squared_error(last_24_actual, first_24_predictions)
mae_24h = mean_absolute_error(last_24_actual, first_24_predictions)
r2_24h = r2_score(last_24_actual, first_24_predictions)
explained_variance_24h = explained_variance_score(last_24_actual, first_24_predictions)

# Output the error metrics for the 24-hour predictions
print("First 24-Hour MSE:", mse_24h)
print("First 24-Hour MAE:", mae_24h)
print("First 24-Hour R-squared:", r2_24h)
print("First 24-Hour Explained Variance:", explained_variance_24h)

# Plot for the first 24-hour predictions vs actuals
plt.figure(figsize=(10, 5))
plt.plot(range(24), last_24_actual, label='Actual', marker='o', color='red')
plt.plot(range(24), first_24_predictions, label='Predicted', marker='x', color='blue', linestyle='--')
plt.title('First 24-Hour Forecast vs Actual')
plt.xlabel('Hour')
plt.ylabel('u Component Value')
plt.xticks(range(24))  # Set x-ticks to show each hour
plt.legend()
plt.show()